SOS2020_ExSOM-C_group12_1553958_01126793
# C) Evaluation Report

todo: intro

## Overview
We trained and loaded 40x20 and 100x60 maps and visualized the results as overlay over U-matrices
We used the following parameters for Neighbourhood Graphs:
* Chainlink Dataset - 40x20
   * k-NN: 1, 2, 3, 4, 20
   * Radius: 0.03, 0.09, 0.15, 0.21, 1
* Chainlink Dataset - 100x60
   * k-NN:
   * Radius:

* 10-Clusters Dataset - 40x20
   * k-NN:
   * Radius:
* 10-Clusters Dataset - 100x60
   * k-NN:
   * Radius:

In [37]:
import numpy as np
import minisom as som
from IPython.core.display import display
from ipywidgets import HBox
from sklearn import datasets, preprocessing

from src.NeighbourhoodGraph import NeighbourhoodGraph
from src.som_vis import SomViz, SOMToolBox_Parse

In [38]:
# Set this parameter to False to run Notebook without training new SOMs
do_training = False


In [39]:
def create_visualizations(sweights, smap_y:int, smap_x:int, idata,
                          color='viridis', interp=False, data_title='Chainlink',
                          k_list = [1,2,3,4],
                          r_list = [0.03,0.09,0.15,0.21],
                          width = 700, height = None,
                          scale_to_mean = False,
                          show_hithist = False):
    # Visualization
    viz = SomViz(sweights, smap_y, smap_x)

    if scale_to_mean: # scale radii to input data
        r_list = np.around(np.mean(idata)*np.array(r_list), decimals=8)

    if height is None: height = width * (smap_y/smap_x)

    # Prepare Neighbourhood Graph
    ng = NeighbourhoodGraph(viz.weights, viz.m, viz.n, input_data=idata)
    traces = []
    for val in k_list:
        traces.append(ng.get_trace(knn=val))
    for val in r_list:
        traces.append(ng.get_trace(radius=val))

    # Prepare underlying visualizations
    umatrix_title = '{}: U-Matrix'.format(data_title)
    hithist_title = '{}: Hit hist'.format(data_title)

    # Show underlying visualizations with Neighbourhood Graph overlay
    for i, trace in enumerate(traces):
        if i < len(k_list):
            title_suffix = " + Neighbourhood Graph ({}-NN)".format(k_list[i])
        else:
            title_suffix = " + Neighbourhood Graph (radius: {})".format(r_list[i-len(k_list)])

        umatrix = viz.umatrix(color=color, interp=interp, title=umatrix_title + title_suffix)
        umatrix.layout.width = width
        umatrix.layout.height = height
        umatrix.add_trace(trace)

        if show_hithist:
            hithist = viz.hithist(idata=idata, color=color, interp=interp, title=hithist_title + title_suffix)
            hithist.layout.width = width
            hithist.layout.height = height
            hithist.add_trace(trace)

            display(HBox([umatrix, hithist]))
        else:
            display(umatrix)

# Chainlink Dataset

In [40]:
# Load and prepare input data
ivec = SOMToolBox_Parse('../input/chainlink/chainlink.vec')
idata, idim, _, _ = ivec.read_weight_file()
idata_chainlink = idata.to_numpy()

## 40x20 Sized Maps

In [41]:
smap_x=40
smap_y=20
weights_path = '../input/chainlink/chainlink4020/chainlink.wgt.gz'
data_title = "Chainlink 40x20"

_k_list = [1, 2, 3, 4, 20]
_r_list = [0.03, 0.09, 0.15, 0.21, 1]

### Train New SOM

In [42]:
if do_training:
    # Train
    smap_chainlink = som.MiniSom(smap_y, smap_x, idata_chainlink.shape[1], sigma=7, learning_rate=0.7)
    %time smap_chainlink.train_random(idata_chainlink, 10000, verbose=False)
    sweights = smap_chainlink._weights.reshape(-1,idim)

    # Visualize
    create_visualizations(sweights,smap_y,smap_x,idata_chainlink,
                          data_title=data_title + " Trained",
                          k_list=_k_list,
                          r_list=_r_list)

### Load Trained SOM

In [43]:
# Load
smap_chainlink_from_file = SOMToolBox_Parse(weights_path)
sweights, sdim, smap_x, smap_y = smap_chainlink_from_file.read_weight_file()

# Fix orientation
sweights = sweights.values.reshape(smap_y,smap_x,sdim)
sweights = np.flip(sweights,0)
sweights = sweights.reshape(-1,sdim)

# Visualize
create_visualizations(sweights,smap_y,smap_x,idata_chainlink,
                      data_title=data_title + " Loaded",
                      k_list=_k_list,
                      r_list=_r_list)

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

## 100x60 Sized Map

In [44]:
smap_x=100
smap_y=60
weights_path = '../input/chainlink/chainlink10060/chainlink.wgt.gz'
data_title = "Chainlink 100x60"

_k_list = [1, 2, 4, 20]
_r_list = [0.03, 0.09, 0.4]

### Train New SOM

In [45]:
# Train
if do_training:
    # Train
    smap_chainlink = som.MiniSom(smap_y, smap_x, idata_chainlink.shape[1], sigma=7, learning_rate=0.7)
    %time smap_chainlink.train_random(idata_chainlink, 10000, verbose=False)
    sweights = smap_chainlink._weights.reshape(-1,idim)

    # Visualize
    create_visualizations(sweights,smap_y,smap_x,idata_chainlink,
                          data_title=data_title + " Trained",
                          k_list=_k_list,
                          r_list=_r_list)

### Load Trained SOM

In [46]:
# Load
smap_chainlink_from_file = SOMToolBox_Parse(weights_path)
sweights, sdim, smap_x, smap_y = smap_chainlink_from_file.read_weight_file()

# Fix orientation
sweights = sweights.values.reshape(smap_y,smap_x,sdim)
sweights = np.flip(sweights,0)
sweights = sweights.reshape(-1,sdim)

# Visualize
create_visualizations(sweights,smap_y,smap_x,idata_chainlink,
                      data_title=data_title + " Loaded",
                      k_list=_k_list,
                      r_list=_r_list)

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

# 10-Clusters Dataset

In [47]:
# Load and prepare input data
ivec = SOMToolBox_Parse('../input/10clusters/10clusters.vec')
idata, idim, _, _ = ivec.read_weight_file()
idata_chainlink = idata.to_numpy()

## 40x20 Sized Maps

In [48]:
smap_x=40
smap_y=20
weights_path = '../input/10clusters/10clusters4020/10clusters.wgt.gz'
data_title = "10-Clusters 40x20"

_k_list = [1, 2, 3, 10]
_r_list = [0.03, 0.09, 0.15, 0.21, 0.5]

### Train New SOM

In [49]:
if do_training:
    # Train
    smap_chainlink = som.MiniSom(smap_y, smap_x, idata_chainlink.shape[1], sigma=7, learning_rate=0.7)
    %time smap_chainlink.train_random(idata_chainlink, 10000, verbose=False)
    sweights = smap_chainlink._weights.reshape(-1,idim)

    # Visualize
    create_visualizations(sweights,smap_y,smap_x,idata_chainlink,
                          data_title=data_title + " Trained",
                          k_list=_k_list,
                          r_list=_r_list)

### Load Trained SOM

In [50]:
# Load
smap_chainlink_from_file = SOMToolBox_Parse(weights_path)
sweights, sdim, smap_x, smap_y = smap_chainlink_from_file.read_weight_file()

# Fix orientation
sweights = sweights.values.reshape(smap_y,smap_x,sdim)
sweights = np.flip(sweights,0)
sweights = sweights.reshape(-1,sdim)

# Visualize
create_visualizations(sweights,smap_y,smap_x,idata_chainlink,
                      data_title=data_title + " Loaded",
                      k_list=_k_list,
                      r_list=_r_list)

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

## 100x60 Sized Map

In [51]:
smap_x=100
smap_y=60
weights_path = '../input/10clusters/10clusters10060/10clusters.wgt.gz'
data_title = "10-Clusters 100x60"

_k_list = [1, 2, 3, 10]
_r_list = [0.03, 0.09, 0.15, 0.21, 0.5]

### Train New SOM

In [52]:
# Train
if do_training:
    # Train
    smap_chainlink = som.MiniSom(smap_y, smap_x, idata_chainlink.shape[1], sigma=7, learning_rate=0.7)
    %time smap_chainlink.train_random(idata_chainlink, 10000, verbose=False)
    sweights = smap_chainlink._weights.reshape(-1,idim)

    # Visualize
    create_visualizations(sweights,smap_y,smap_x,idata_chainlink,
                          data_title=data_title + " Trained",
                          k_list=_k_list,
                          r_list=_r_list)

### Load Trained SOM

In [53]:
# Load
smap_chainlink_from_file = SOMToolBox_Parse(weights_path)
sweights, sdim, smap_x, smap_y = smap_chainlink_from_file.read_weight_file()

# Fix orientation
sweights = sweights.values.reshape(smap_y,smap_x,sdim)
sweights = np.flip(sweights,0)
sweights = sweights.reshape(-1,sdim)

# Visualize
create_visualizations(sweights,smap_y,smap_x,idata_chainlink,
                          data_title=data_title + " Loaded",
                      k_list=_k_list,
                      r_list=_r_list)


KeyboardInterrupt: 